In [2]:
import pandas as pd
import numpy as np
import pymc as pm

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")
flipped

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/3063244417.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")


,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629274,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629275,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629276,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629277,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [4]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_time', 'new_x', 'new_y', 'chip_side',
       'end_time', 'Right_End', 'Left_End', 'Flip'],
      dtype='object')

In [5]:
frames_per_play = flipped[flipped['pff_positionLinedUp']=='QB'].groupby('uniqueplayId', as_index=False).size().rename(columns={'size':'play_length_frames'})

In [6]:
frames_per_play

,uniqueplayId,play_length_frames
0,202109090097,43
1,202109120056,37
2,202109120163,32
3,202109120288,27
4,202109120578,27
...,...,...
5693,20211025003536,63
5694,20211025003660,33
5695,20211025003684,53
5696,20211025003904,37


In [7]:
flipped = flipped.merge(frames_per_play, on='uniqueplayId', how='left')

In [10]:
flipped['play_length_frames'].min()

19

In [12]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_time', 'new_x', 'new_y', 'chip_side',
       'end_time', 'Right_End', 'Left_End', 'Flip', 'play_length_frames'],
      dtype='object')

In [93]:
def name_func(pos):
    temp = {'nflId':pos+'_nflId',
     'jerseyNumber':pos+'_jerseyNumber',
     'team':pos+'_team', 
     'x':pos+'_x', 
     'y':pos+'_y',
     's':pos+'_s', 
     'a':pos+'_a', 
     'dis':pos+'_dis', 
     'o':pos+'_o', 
     'dir':pos+'_dir', 
     'pff_role':pos+'_pff_role', 
     'pff_positionLinedUp':pos+'_pff_positionLinedUp',
     'pff_nflIdBlockedPlayer':pos+'_pff_nflIdBlockedPlayer', 
     'pff_blockType':pos+'_pff_blockType', 
     'new_x':pos+'_new_x', 
     'new_y':pos+'_new_y', 
     'Flip':pos+'_Flip'}
    return temp

In [94]:
qb = flipped[(flipped['pff_positionLinedUp']=='QB')]
qb = qb.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [95]:
qb_flip = flipped[(flipped['pff_positionLinedUp']=='QB_flip')]
qb_flip = qb_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('QB_flip')).reset_index(drop=True)

In [96]:
ball = flipped[(flipped['pff_positionLinedUp']=='ball')]
ball = ball.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)

In [97]:
ball_flip = flipped[(flipped['pff_positionLinedUp']=='ball_flip')]
ball_flip = ball_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('ball_flip')).reset_index(drop=True)


In [98]:
rt = flipped[(flipped['pff_positionLinedUp']=='RT')]
rt = rt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('RT')).reset_index(drop=True)

In [99]:
lt = flipped[(flipped['pff_positionLinedUp']=='LT')]
lt = lt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('LT')).reset_index(drop=True)

In [100]:
le = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)]
le = le.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('LE')).reset_index(drop=True)

In [101]:
re = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==0)]
re = re.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('RE')).reset_index(drop=True)

In [102]:
play = flipped[(flipped['pff_positionLinedUp']=='QB')][['uniqueplayId', 'football_x', 'football_y', 
                                                        'gameId', 'playId', 'frameId', 'time', 'playDirection', 
                                                        'event', 'ball_snap_time', 'end_time', 'Right_End', 
                                                        'Left_End', 'play_length_frames']].reset_index(drop=True)

In [103]:
final = play.merge(qb, on=['uniqueplayId', 'frameId'], how='left').merge(qb_flip, on=['uniqueplayId', 'frameId'], how='left')

In [104]:
final = final.merge(ball, on=['uniqueplayId', 'frameId'], how='left').merge(ball_flip, on=['uniqueplayId', 'frameId'], how='left')

In [105]:
final = final.merge(rt, on=['uniqueplayId', 'frameId'], how='left').merge(lt, on=['uniqueplayId', 'frameId'], how='left')

In [106]:
final = final.merge(re, on=['uniqueplayId', 'frameId'], how='left').merge(le, on=['uniqueplayId', 'frameId'], how='left')

In [107]:
final

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_time,...,LE_dis,LE_o,LE_dir,LE_pff_role,LE_pff_positionLinedUp,LE_pff_nflIdBlockedPlayer,LE_pff_blockType,LE_new_x,LE_new_y,LE_Flip
0,202109090097,41.56,23.92,2021090900,97,1,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.01,276.51,290.12,Pass Rush,LILB,NaN,NaN,2.15,2.04,1.0
1,202109090097,41.56,23.92,2021090900,97,2,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.00,276.51,289.82,Pass Rush,LILB,NaN,NaN,2.15,2.04,1.0
2,202109090097,41.56,23.92,2021090900,97,3,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.01,277.16,296.04,Pass Rush,LILB,NaN,NaN,2.15,2.03,1.0
3,202109090097,41.56,23.92,2021090900,97,4,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.02,277.16,308.81,Pass Rush,LILB,NaN,NaN,2.14,2.01,1.0
4,202109090097,41.56,23.92,2021090900,97,5,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.01,277.16,325.47,Pass Rush,LILB,NaN,NaN,2.13,2.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214235,20211025003926,85.95,23.68,2021102500,3926,45,2021-10-26 03:14:08,left,None,2021-10-26 03:14:05,...,0.19,113.22,26.59,Pass Rush,LOLB,NaN,NaN,4.83,3.77,1.0
214236,20211025003926,85.95,23.68,2021102500,3926,46,2021-10-26 03:14:08,left,None,2021-10-26 03:14:05,...,0.19,113.22,23.83,Pass Rush,LOLB,NaN,NaN,4.91,3.94,1.0
214237,20211025003926,85.95,23.68,2021102500,3926,47,2021-10-26 03:14:09,left,None,2021-10-26 03:14:05,...,0.18,113.88,21.57,Pass Rush,LOLB,NaN,NaN,4.98,4.11,1.0
214238,20211025003926,85.95,23.68,2021102500,3926,48,2021-10-26 03:14:09,left,None,2021-10-26 03:14:05,...,0.18,115.29,18.94,Pass Rush,LOLB,NaN,NaN,5.05,4.28,1.0


In [120]:
unflipped = pd.read_csv('https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true')

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2957017977.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  unflipped = pd.read_csv('https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true')


In [141]:
play = final[final['uniqueplayId']==20210919023392]

In [142]:
play

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_time,...,LE_dis,LE_o,LE_dir,LE_pff_role,LE_pff_positionLinedUp,LE_pff_nflIdBlockedPlayer,LE_pff_blockType,LE_new_x,LE_new_y,LE_Flip
40963,20210919023392,32.31,29.83,2021091902,3392,3,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.00,354.67,338.24,Pass Rush,LEO,NaN,NaN,0.22,6.83,1.0
40964,20210919023392,32.31,29.83,2021091902,3392,4,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.02,350.51,300.50,Pass Rush,LEO,NaN,NaN,0.20,6.83,1.0
40965,20210919023392,32.31,29.83,2021091902,3392,5,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.02,347.07,280.39,Pass Rush,LEO,NaN,NaN,0.18,6.82,1.0
40966,20210919023392,32.31,29.83,2021091902,3392,6,2021-09-19 19:42:46,right,ball_snap,2021-09-19 19:42:46,...,0.03,345.35,282.93,Pass Rush,LEO,NaN,NaN,0.15,6.81,1.0
40967,20210919023392,32.31,29.83,2021091902,3392,7,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.07,344.11,286.61,Pass Rush,LEO,NaN,NaN,0.08,6.79,1.0
40968,20210919023392,32.31,29.83,2021091902,3392,8,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.10,339.02,289.73,Pass Rush,LEO,NaN,NaN,0.01,6.76,1.0
40969,20210919023392,32.31,29.83,2021091902,3392,9,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.16,330.54,289.16,Pass Rush,LEO,NaN,NaN,0.16,6.70,1.0
40970,20210919023392,32.31,29.83,2021091902,3392,10,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.21,328.35,290.53,Pass Rush,LEO,NaN,NaN,0.36,6.63,1.0
40971,20210919023392,32.31,29.83,2021091902,3392,11,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.26,327.30,291.00,Pass Rush,LEO,NaN,NaN,0.60,6.53,1.0
40972,20210919023392,32.31,29.83,2021091902,3392,12,2021-09-19 19:42:46,right,None,2021-09-19 19:42:46,...,0.32,325.71,291.77,Pass Rush,LEO,NaN,NaN,0.90,6.41,1.0


In [143]:
play[['RE_dir', 'RE_o', 'LT_dir', 'LT_o']]

,RE_dir,RE_o,LT_dir,LT_o
40963,243.93,249.78,301.27,78.06
40964,257.76,244.89,293.78,74.71
40965,256.24,241.87,292.08,72.57
40966,266.51,234.69,293.86,69.29
40967,268.09,230.35,294.26,65.97
40968,267.54,244.42,293.68,60.94
40969,267.69,241.57,292.99,57.10
40970,266.23,238.02,293.10,52.38
40971,264.89,228.60,290.40,49.58
40972,263.17,237.27,290.74,48.25


Before classifying:
    Remove frames before ball is snapped and after it is thrown
    Remove tackle and end where the end's pff_role is Coverage
    First check: Remove plays where end moves into the inside (stunt)
    Remove plays where the distance between end and tackle is large directly after ball snap (not against each other)

### Classifying
Clustering into two groups
Predictor variables:
    Percentage that tackle/end are looking at each other
    Percentage that Euclidian distance is less than 0.5 (0.75)
    Total distance traveled of defensive end
    Max speed during rush